<a href="https://colab.research.google.com/github/monicman/dataengineering/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyspark

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import urllib
from urllib.request import urlopen   
import io
import requests
from zipfile import ZipFile
import os

In [0]:
spark = SparkSession.builder.getOrCreate()

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    print("error here")
    df = pd.read_csv(filename,sep='\t')
    print("no actually here")
    return spark.createDataFrame(df)

In [0]:
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    print("error here")
    df = pd.read_csv(filename,sep='\t')
    df.iloc[:, 3:6] = df.iloc[:, 3:6].apply(pd.to_datetime, errors='coerce', format='%Y/%m/%d')
    print("no actually here")
    myschema = StructType([StructField('REGISTER_NAME', StringType(), True),
                         StructField('BN_NAME', StringType(), True),
                         StructField('BN_STATUS', StringType(), True),
                         StructField('BN_REG_DT', DateType(), True),
                         StructField('BN_CANCEL_DT', DateType(), True),
                         StructField('BN_RENEW_DT', DateType(), True),
                         StructField('BN_STATE_NUM', StringType(), True),
                         StructField('BN_STATE_OF_REG', StringType(), True),
                         StructField('BN_ABN', DoubleType(), True)])
    return spark.createDataFrame(df, schema=myschema)

In [92]:
ipgod101 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/e5cbeafc-5fb3-4dfd-bd22-afe81b6ab1e1/download/ipgod101.csv'
ipgod102 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/846990df-db42-4ad7-bbd6-567fd37a2797/download/ipgod102.csv'  
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'

staging_ipgod101 = ingest_files(ipgod101,'ipgod101.csv')
staging_ipgod102 = ingest_files(ipgod102,'ipgod102.csv')
staging_abr = ingest_files(ABN,'zipFile.zip')

error here
no actually here


Step 1: Get ABN Data and stage it in spark

In [0]:
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'

url = urlopen(URL)
output = open('zipFile.zip', 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

df = pd.read_csv('zipFile.zip',sep='\t')   # pandas version 0.18.1 takes zip files       


In [0]:
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'


'zip'

In [0]:
df.head()

,REGISTER_NAME,BN_NAME,BN_STATUS,BN_REG_DT,BN_CANCEL_DT,BN_RENEW_DT,BN_STATE_NUM,BN_STATE_OF_REG,BN_ABN
2612578,BUSINESS NAMES,VEGETABLE OIL WHOLESALERS AUSTRALIA,Registered,17/02/2003,NaN,17/02/2021,B1675364R,VIC,3.806964e+10
1566829,BUSINESS NAMES,Jo & Adam's Takeaway,Registered,21/12/2016,NaN,21/12/2020,NaN,NaN,1.991220e+10
560846,BUSINESS NAMES,BYFORD FUGITIVE AUTO PARTS & SERVICE,Registered,27/06/2002,NaN,27/06/2020,BN09531221,WA,7.810624e+10
2206955,BUSINESS NAMES,Toowoomba Scrap Cars and Commercials,Registered,08/05/2018,NaN,08/05/2022,NaN,NaN,2.136973e+10
1816358,BUSINESS NAMES,Over the Way,Registered,10/06/2013,NaN,10/06/2019,NaN,NaN,8.620810e+10


In [0]:
ipgod101.head()

,australian_appl_no,application_year,sealing_year,applicant_count,australian,foreign,entity,non_entity,patent_status_type,patent_type,primary_ipc_mark_value
0,2018904968,2018,NaN,1.0,True,False,False,True,FILED,Provisional,NaN
1,2017215797,2018,NaN,1.0,False,True,True,False,FILED,Standard,E06B 9/00
2,2017216336,2018,NaN,1.0,False,True,True,False,FILED,Standard,A61M 31/00
3,2017216324,2018,NaN,1.0,False,True,True,False,FILED,Standard,H04W 74/08
4,2017216313,2018,NaN,1.0,False,True,True,False,FILED,Standard,H01L 29/06


In [0]:
ipgod102[ipgod102['ipa_id'] == 322357.0]

,australian_appl_no,ipa_id,country,australian,entity,name,cleanname,corp_desg,state,postcode,lat,lon,sa2_code,sa2_name,lga_code,lga_name,gcc_name,elect_div,abn,acn
1260798,2012905138,322357.0,AU,True,True,Elevare Energy IP Pty Ltd,ELEVARE ENERGY IP,PTY LTD,QLD,4034.0,-27.357,153.018,302021027.0,Aspley,31000.0,Brisbane (C),Greater Brisbane,Petrie,NaN,NaN
1290230,2013350321,322357.0,AU,True,True,Elevare Energy IP Pty Ltd,ELEVARE ENERGY IP,PTY LTD,QLD,4034.0,-27.357,153.018,302021027.0,Aspley,31000.0,Brisbane (C),Greater Brisbane,Petrie,NaN,NaN
1449069,2018201166,322357.0,AU,True,True,Elevare Energy IP Pty Ltd,ELEVARE ENERGY IP,PTY LTD,QLD,4034.0,-27.357,153.018,302021027.0,Aspley,31000.0,Brisbane (C),Greater Brisbane,Petrie,NaN,NaN
1464868,2019900440,322357.0,AU,True,True,Elevare Energy IP Pty Ltd,ELEVARE ENERGY IP,PTY LTD,QLD,4064.0,-27.474,153.004,305041135.0,Paddington - Milton,31000.0,Brisbane (C),Greater Brisbane,Brisbane,NaN,NaN
